## Importation des librairies

In [1]:
import pandas as pd
import tkinter as tk
from tkinter import *
import matplotlib.pyplot as plt
import math
import rpack
import random
import numpy as np
import sys

np.set_printoptions(threshold=sys.maxsize)

In [2]:
from IPython.core.display import display, HTML

display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
class App(tk.Tk):
    def __init__(self):
        super().__init__()

        self.title('Window')
        self.h = 1000
        self.w = 2000
        self.geometry(str(self.w)+'x'+str(self.h))
   
    def getH(self):
        return self.h
    
    def getW(self):
        return self.w
    
    def quit(self):
        self.destroy()

In [134]:
class Piece():
    def __init__(self):
        self.x = 0
        self.y = 0
        self.h = 0
        self.w = 0
        self.pct = 0
        self.oldwh = []
        
        self.tlc = []
        self.trc = []
        self.blc = []
        self.brc = []
        
        self.center = []
        
    def setX(self,x):
        self.x = x
        
    def setY(self,y):
        self.y = y
        
    def setW(self,w):
        self.w = w
        
    def setH(self,h):
        self.h = h
    
    def setPct(self, pct):
        self.pct = pct        
    
    def setTlc(self, tlc):
        self.tlc = tlc
    
    def getTlc(self):
        self.tlc = [self.x, self.y]
        return self.tlc
    
    def getTrc(self):
        self.trc = [self.x+self.w, self.y]
        return self.trc
    
    def getBlc(self):
        self.blc = [self.x, self.y+self.h]
        return self.blc
    
    def getBrc(self):
        self.brc = [self.x+self.w, self.y+self.h]
        return self.brc
    
    def getCenter(self):
        tlc = self.getTlc()
        self.center = [tlc[0]+(self.w/2), tlc[1]+(self.h/2)]
        return self.center
    
    def appendOldWH(self):
        self.oldwh.append([self.w,self.h])
        
    def getOldWH(self):
        return self.oldwh
         
    
class Chambre(Piece):
    def __init__(self):
        Piece.__init__(self)
        self.h = 100
        self.w = 100
        self.pct = 15
        self.pctdef = 15
        self.num = 1
        self.col = "orange"
        self.prio = 2
    
    def __str__(self):
        return "Chambre"
        
class Cuisine(Piece):
    def __init__(self):
        Piece.__init__(self)
        self.h = 100
        self.w = 100
        self.pct = 15
        self.pctdef = 15
        self.num = 2
        self.col = "green"
        self.prio = 3
        
    def __str__(self):
        return "Cuisine"
        
class Sam(Piece):
    def __init__(self):
        Piece.__init__(self)
        self.h = 100
        self.w = 100
        self.pct = 15
        self.pctdef = 15
        self.num = 3
        self.col = "purple"
        self.prio = 1
        
    def __str__(self):
        return "Salle à manger"
    
class Sdb(Piece):
    def __init__(self):
        Piece.__init__(self)
        self.h = 100
        self.w = 100
        self.pct = 10
        self.pctdef = 10
        self.num = 4
        self.col = "blue"
        self.prio = 4
    
    def __str__(self):
        return "Salle de bain"
        
class Toilettes(Piece):
    def __init__(self):
        Piece.__init__(self)
        self.h = 100
        self.w = 100
        self.pct = 5
        self.pctdef = 5
        self.num = 5
        self.col = "grey"
        self.prio = 5
        
    def __str__(self):
        return "Toilettes"
        
class Rangement(Piece):
    def __init__(self):
        Piece.__init__(self)
        self.h = 100
        self.w = 100
        self.pct = 5
        self.pctdef = 5
        self.num = 6
        self.col = "maroon"
    
    def __str__(self):
        return "Rangement"
    
class Buanderie(Piece):
    def __init__(self):
        Piece.__init__(self)
        self.h = 100
        self.w = 100
        self.pct = 5
        self.pctdef = 5
        self.num = 7
        self.col = "cyan"
        
    def __str__(self):
        return "Buanderie"

class Entree(Piece):
    def __init__(self):
        Piece.__init__(self)
        self.h = 100
        self.w = 100
        #self.pct = 15
        self.num = 8
        self.col = "yellow"
    
    def __str__(self):
        return "EntréeS"    

def tabModulo(n):
    n = n
    l = []
    
    #print(n)
    
    for i in range (1,n+1):
        if n%i == 0 and i != 1 and i != n:
            l.append(i)
            
    nl = l[::-1]
    tab = pd.DataFrame(l, columns = [1])
    tab[2] = nl
    #display(tab)
    
    return tab

In [135]:
def uniq(lst):
    last = object()
    for item in lst:
        if item == last:
            continue
        yield item
        last = item

def sort_and_deduplicate(l):
    return list(uniq(sorted(l, reverse=True)))

In [263]:
class Appartement():
    def __init__(self, app, h, w):
        self.app = app
        self.x = 150
        self.y = 50
        self.h = h
        self.w = w
        self.listePieces = []
        self.prio = 1000
        
        self.tlc = [self.x, self.y, "TLC"]
        self.trc = [self.x+self.w, self.y, "TRC"]
        self.blc = [self.x, self.y+self.h, "BLC"]
        self.brc = [self.x+self.w, self.y+self.h, "BRC"]
        self.listeCoinsFinale = []
        self.listeCoinsTotale = []
        self.listeCoinsAppart = [[self.tlc],[self.trc],[self.brc],[self.blc]]
        self.listeCoinsRemoved = []
        
        self.ent = Entree()
        
        self.mat = np.zeros((self.w, self.h))
        
        self.can = tk.Canvas(self.app, bg = "white", height = app.getH(), width = app.getW())
        
    def afficherAppartement(self):
        self.can.create_rectangle(self.x, self.y, 
                             self.x+self.w, self.y+self.h)
        self.can.create_rectangle(self.x+700, self.y, 
                             self.x+700+self.w, self.y+self.h)
        
                             #fill = "white" )
        #can.create_polygon(self.x, self.y,
        #                   (self.x+self.w)-(self.x+self.w)/4, self.y,
        #                   (self.x+self.w)-(self.x+self.w)/4, self.y+(self.x+self.w)/6,
        #                   self.x+self.w, self.y+(self.x+self.w)/6,
        #                   self.x+self.w, self.y + self.h,
        #                   self.x, self.y + self.h, 
        #                   outline = "black",
        #                   fill = "white" )
        
        self.ent.setX((self.x+(self.w)/2)-25)
        self.ent.setY(self.y)
        self.ent.setW(50)
        self.ent.setH(self.y)
        tlc = self.ent.getTlc()
        brc = self.ent.getBrc()
        #self.can.create_rectangle(tlc[0],tlc[1],brc[0],brc[1], outline = "black")
        
        self.can.create_line((self.x+(self.w)/2)-17, self.y,(self.x+(self.w)/2)+17, self.y, fill = "white")
        self.can.create_line((self.x+700+(self.w)/2)-17, self.y,(self.x+700+(self.w)/2)+17, self.y, fill = "white")
        self.can.pack()
    
    def afficherGrille(self):
        self.dimcase = math.sqrt((5*(self.h*self.w))/100)
        self.nbcasesh = int(self.h/self.dimcase)
        self.nbcasesw = int(self.w/self.dimcase)
        #print(self.dimcase)
        #print(self.nbcasesh)
        #print(self.nbcasesw)
        for i in range(self.nbcasesh):
            for j in range(self.nbcasesw):
                #print(i)
                self.can.create_rectangle(self.x+j*self.dimcase, self.y+i*self.dimcase, self.x+(j+1)*self.dimcase, self.y+(i+1)*self.dimcase, outline = 'red')
        
        self.can.pack()
    
    def drawRoom(self, tlc, brc, piece, k):
        self.can.create_rectangle(tlc[0]+k,tlc[1],brc[0]+k,brc[1], outline = piece.col) #, fill = "white")
        self.can.create_text(tlc[0]+k+(piece.w/2),tlc[1]+(piece.h/2),fill="black",font="Times 8",text=f"{piece.__str__()}")
    
    def dimensionRoom(self, piece):
        piece.setPct(piece.pctdef)
        piece.setX(self.x)
        piece.setY(self.y)
        
        totalpct = 100

        if piece.pct == 5:
            rndpct = random.randrange(0,2,2)
        else:
            rndpct = random.randrange(-5,6,5)

        if totalpct > 15:
            piece.setPct(piece.pct+rndpct)

        totalpct = totalpct-(piece.pct+rndpct)

        #print(piece.__str__()+" "+str(piece.pct))
        #print(totalpct)

        dimcase = int((piece.pct*(self.h*self.w))/100)
        tab = tabModulo(dimcase)

        for i in range(2):
            dimliste = [self.w, self.h]
            indexModuloLow = tab[tab[i+1] < 100].index
            indexModuloHigh = tab[tab[i+1] > dimliste[i]].index
            tab.drop(indexModuloLow, inplace = True)
            tab.drop(indexModuloHigh, inplace = True)

        #display(tab)
        
        tabOldWH = piece.getOldWH()
        
        rnddim = random.choice(tab.values.tolist())
        
        while rnddim in tabOldWH:
            print(rnddim, tabOldWH)
            rnddim = random.choice(tab.values.tolist())
               
        piece.setW(rnddim[0])
        piece.setH(rnddim[1])
            
        
        #return piece
    
    def createRooms(self):      
        chb = Chambre()
        csn = Cuisine()
        sdb = Sdb()
        sam = Sam()
        tlt = Toilettes()
        rng = Rangement()
        bdr = Buanderie()
        
        #self.listePieces = [chb,csn,sdb,sam,tlt,rng,bdr]
        self.listePieces = [chb,csn,sdb,sam,tlt]
        random.shuffle(self.listePieces)
        
        idx = 0
        sumW = 0
        for piece in self.listePieces:
            self.dimensionRoom(piece)
            sumW = sumW+piece.w
            tlc = [piece.x+(sumW-piece.w),piece.y+450]
            brc = [piece.x+sumW, piece.y+450+piece.h]
            
            self.drawRoom(tlc, brc, piece,0)
            
            idx += 1
        
        self.can.pack()
        
    def placeRooms(self):
        L = self.listePieces
        placed = []
        done = 0
        sumW = 0
        sumH = 0
        k = 0
        
        while len(placed) != len(L):
            for piece in L:
                if self.x+piece.w < (self.x+(self.w)/2)-17 and done == 0:
                    piece.setX(self.x)
                    piece.setY(self.y)
                    
                    self.drawRoom(piece.getTlc(), piece.getBrc(), piece,0)
                                        
                    done = 1
                    sumW = piece.w
                    sumH = piece.h
                    placed.append(piece)
                    
                    #self.majMatrice(piece)
                    #self.app.update()
                    #input("Press Enter to continue...")
                    
                if piece in placed:
                    #print(placed)
                    for piece2 in L:
                        #print((self.x+(self.w)/2)-17)
                        if piece2.__str__() != piece.__str__() and sumH + piece2.h <= self.h and piece2 not in placed:
                            tlc = piece.getBlc()
                            piece2.setX(tlc[0])
                            piece2.setY(tlc[1])
                            
                            self.drawRoom(piece2.getTlc(), piece2.getBrc(), piece2, 0)
                            
                            placed.append(piece2)
                            sumH = sumH + piece2.h
                            
                            #self.majMatrice(piece2)
                            #self.app.update()
                            
                            break
                            
                    for piece2 in L:
                        if piece.y != self.y:
                            lim = self.w
                        else:
                            lim = ((self.w)/2)-17
                            
                        if piece2.__str__() != piece.__str__() and sumW + piece2.w <= lim and piece2 not in placed:
                            tlc = piece.getTrc()
                            piece2.setX(tlc[0])
                            piece2.setY(tlc[1])

                            self.drawRoom(piece2.getTlc(), piece2.getBrc(), piece2, 0)

                            placed.append(piece2)
                            sumW = sumW + piece2.w
                            
                            #self.majMatrice(piece2)
                            #self.app.update()
                            
                            break
                            
                    print("ite "+str(k))
                    for p in placed:
                        print(p.__str__())
                        
                    k+=1
                    
            placed = L 
                
            self.can.pack()
    
    def f(self, piece, listeCoins, placed, listeCoinsTotale, listeCoinsRemoved):
            listeCoinsPotentiels = []
            multipleToRemove = []
            
            if len(listeCoins) == 0:
                piece.appendOldWH()
                self.dimensionRoom(piece)
                #self.app.update()
                print("\n\nREDIMENSION !!!!!!!!!!!!!!!!!")
                for elem in listeCoinsTotale:
                    if elem not in listeCoinsRemoved:
                        listeCoins.append(elem)
            
            print("\n"+piece.__str__())  
            print(f"TOTALE : {listeCoinsTotale}")
            print(f"COINS : {listeCoins}")
            print(f"REMOVED : {listeCoinsRemoved}")
            coin = random.choice(listeCoins)
            
            #print(coin)
            #print([coin[0]-self.x,coin[1]-self.y,coin[2]])
            
            if coin[2] == "TLC":
                piece.setX(coin[0])
                piece.setY(coin[1])
                
                trc = piece.getTrc() + ['TLC']
                blc = piece.getBlc() + ['TLC']
                
                listeCoinsPotentiels.append(trc)
                listeCoinsPotentiels.append(blc)

            if coin[2] == "TRC":
                piece.setX(coin[0]-piece.w)
                piece.setY(coin[1])
                
                tlc = piece.getTlc() + ['TRC']
                brc = piece.getBrc() + ['TRC']

                listeCoinsPotentiels.append(tlc)
                listeCoinsPotentiels.append(brc)

            if coin[2] == "BLC":
                piece.setX(coin[0])
                piece.setY(coin[1]-piece.h)
                
                tlc = piece.getTlc() + ['BLC']
                brc = piece.getBrc() + ['BLC']
                
                listeCoinsPotentiels.append(tlc)
                listeCoinsPotentiels.append(brc)

            if coin[2] == "BRC":
                piece.setX(coin[0]-piece.w)
                piece.setY(coin[1]-piece.h)
                
                trc = piece.getTrc() + ['BRC']
                blc = piece.getBlc() + ['BRC']
                
                listeCoinsPotentiels.append(trc)
                listeCoinsPotentiels.append(blc)
            
            ptrc, ptlc, pbrc, pblc = piece.getTrc(),piece.getTlc(),piece.getBrc(),piece.getBlc()
            listeCoinsPiece = [ptrc, ptlc, pbrc, pblc]
            horsChamps = 0
            superposition = 0
            superpositionplaced = 0
            remover = 0
            
            for coinpiece in listeCoinsPiece:
                for coinliste in listeCoins:
                    if coinpiece[0] == coinliste[0] and coinpiece[1] == coinliste[1]:
                        print(f"Removed corner : {coinliste}")
                        listeCoins.remove(coinliste)
                        remover += 1
                        multipleToRemove.append(coinliste)
                if coinpiece[0] < self.x or coinpiece[0] > self.x+self.w or coinpiece[1] < self.y or coinpiece[1] > self.y+self.h:
                    horsChamps = 1
            
            #print(placed)
            for placedpiece in placed:
                pptrc, pptlc, ppbrc, ppblc = placedpiece.getTrc(),placedpiece.getTlc(),placedpiece.getBrc(),placedpiece.getBlc()
                listeppcoins = [pptrc, pptlc, ppbrc, ppblc]
                center = piece.getCenter()
                if (center[0] >= pptlc[0] and center[0] <= ppbrc[0] and center[1] >= pptlc[1] and center[1] <= ppbrc[1]):
                    superposition = 1
                    break
                if (ptlc[0]+1 > pptlc[0] and pbrc[0]-1 < ppbrc[0] and ptlc[1]+1 > pptlc[1] and pbrc[1]-1 < ppbrc[1]):
                    superposition = 1
                    self.drawRoom([ptlc[0]+1,ptlc[1]+1],[pbrc[0]-1,pbrc[1]-1], piece, 0)
                    break
                    
                for coinpiece in listeCoinsPiece:
                    if (coinpiece[0] > pptlc[0] and coinpiece[0] < ppbrc[0] and coinpiece[1] > pptlc[1] and coinpiece[1] < ppbrc[1]):
                        superposition = 1
                    
                for coinpp in listeppcoins:
                    if (coinpp[0] > ptlc[0] and coinpp[0] <= pbrc[0] and coinpp[1] >= ptlc[1] and coinpp[1] < pbrc[1]):
                        superpositionplaced = 1
                        
            if horsChamps == 1 or superposition == 1 or superpositionplaced == 1:
                print(f"hors champs = {horsChamps}, superposition = {superposition}, superpositionPlaced = {superpositionplaced}")
                if remover == 0:
                    listeCoins.remove(coin)
                print("\nREITERATION")
                self.f(piece, listeCoins, placed, listeCoinsTotale, listeCoinsRemoved)
                listeCoins.append(coin)
            else:
                if remover == 0:
                    listeCoins.remove(coin)
                    listeCoinsRemoved.append(coin)
                else:
                    for coin in multipleToRemove:
                        listeCoinsRemoved.append(coin)
                self.drawRoom(piece.getTlc(), piece.getBrc(), piece, 0)
                placed.append(piece)
                for coinp in listeCoinsPotentiels:
                    listeCoins.append(coinp)
                    listeCoinsTotale.append(coinp)
                    self.listeCoinsAppart.append([coinp, piece])
                print(piece.__str__()+" placée")
            #self.majMatrice(piece)
            
            #print(f"REMOVED : {listeCoinsRemoved}")
            '''
            cotes = [self.x, self.y, self.w+self.x, self.h+self.y]
            
            for nb in cotes:
                for placedpiece in placed:
                    tlc, brc = placedpiece.getTlc(), placedpiece.getBrc()
                    mindiff = 0
                    for placedpiece2 in placed:
                        tlc2, brc2 = placedpiece.getTlc(), placedpiece.getBrc()
                        if placedpiece is not placedpiece2:
                            if placedpiece.tlc[0] == nb and placedpiece.tlc[0] == placedpiece.tlc2[0] and placedpiece.tlc[1]-:
                                mindiff = 
                        
            '''
            
            self.listeCoinsTotale = listeCoinsTotale
            self.listeCoinsRemoved = listeCoinsRemoved
            #self.listeCoinsPiece = listeCoinsPiece
            
    def placeRooms2(self):
        #entTlc = self.ent.getTlc() + ['TRC']
        #entTrc = self.ent.getTrc() + ['TLC']
        #entBlc = self.ent.getBlc() + ['BLC']
        #entBrc = self.ent.getBrc() + ['BRC']
        placed = [self.ent]
        listeCoins = [self.tlc,self.trc,self.brc,self.blc]
        listeCoinsTotale = [self.tlc,self.trc,self.brc,self.blc]
        listeCoinsRemoved = []
        
        
        for piece in self.listePieces:
            self.f(piece, listeCoins, placed, listeCoinsTotale, listeCoinsRemoved)
            
        for elem in self.listeCoinsTotale:
            if elem not in self.listeCoinsRemoved:
                self.listeCoinsFinale.append(elem)
              
        #print(self.listeCoinsFinale)
        
        tabCoinsPiece = pd.DataFrame(self.listeCoinsAppart)
        tabCoinsRemoved = pd.DataFrame([self.listeCoinsRemoved]).transpose()
        tabCoinsPiece[3] = None
        #tabCoinsPiece = tabCoinsPiece[tabCoinsPiece[0] not in self.listeCoinsRemoved]
        listeTabs = [tabCoinsPiece,tabCoinsRemoved]
        display(tabCoinsPiece)
        
        for tab in listeTabs:
            i = 0
            tab['corner'] = None
            tab['cornertype'] = 0
            tab['x_corner'] = 0
            tab['y_corner'] = 0
            for elem in tab[0]:
                print(elem)
                tab['corner'][i] = [elem[0],elem[1]]
                tab['x_corner'][i] = elem[0]
                tab['y_corner'][i] = elem[1]
                tab['cornertype'][i] = elem[2]
                
                i += 1
            tab.drop(0, inplace = True, axis = 1)
        
        display(tabCoinsPiece)
        display(tabCoinsRemoved)
        #tabCoinsPieceWithoutRemoved = pd.concat([tabCoinsPiece,tabCoinsRemoved], join = "outer", ignore_index = True, name = 'corner', axis = 1)
        tabCoinsPieceWithoutRemoved = pd.merge(tabCoinsPiece,tabCoinsRemoved, how="outer", on=["x_corner","y_corner"])
        display(tabCoinsPieceWithoutRemoved)
        tabCoinsPieceWithoutRemoved = tabCoinsPieceWithoutRemoved[tabCoinsPieceWithoutRemoved['cornertype_y'].isna()].drop(columns = ["x_corner","y_corner","corner_y","cornertype_y"])
        tabCoinsPieceWithoutRemoved[1] = tabCoinsPieceWithoutRemoved[1].fillna(self)
        tabCoinsPieceWithoutRemoved['prio'] = None
        tabCoinsPieceWithoutRemoved = tabCoinsPieceWithoutRemoved.reset_index().drop('index',axis = 1)
        
        idx = 0
        for elem in tabCoinsPieceWithoutRemoved[1]:
            print(idx,elem,elem.prio)
            tabCoinsPieceWithoutRemoved['prio'][idx] = elem.prio
            idx += 1
        
        tabCoinsPieceWithoutRemoved = tabCoinsPieceWithoutRemoved.sort_values(by='prio')
        display(tabCoinsPieceWithoutRemoved)
        
        doneList = []
        
        for piece, done, coin, typecoin, prio in tabCoinsPieceWithoutRemoved.itertuples(index=False):
            diff = 0
            for piece2, done2, coin2, typecoin2, prio2 in tabCoinsPieceWithoutRemoved.itertuples(index=False):
                if coin != coin2 and piece != piece2:
                    #print(piece, piece2)
                    #print(f"{coin[1]+coin2[1]} > {piece.h+piece2.h}")
                    if coin[0] in [self.tlc[0],self.brc[0]] and coin[0] == coin2[0] and coin not in doneList and coin2 not in doneList: #and ((coin[1]+coin2[1])>(piece.h+piece2.h) or (piece == 'Appartement' or piece2 == 'Appartement'))
                        print('VERTICAL')
                        diff = abs(coin2[1]-coin[1])
                        print(piece,piece2,diff)
                        doneList.append(coin)
                        if prio < prio2:
                            if typecoin == 'TLC' or typecoin == 'TRC':
                                print(f'Tlc {piece.getBlc()}')
                                print(f'Trc {piece.getBrc()}')
                                piece.setH(piece.h + diff)
                                print(f'Tlc {piece.getBlc()}')
                                print(f'Trc {piece.getBrc()}')
                            if typecoin == 'BLC' or typecoin == 'BRC':
                                piece.setY(piece.y - diff)
                                piece.setH(piece.h + diff)
                        else:
                            if typecoin2 == 'TLC' or typecoin2 == 'TRC':
                                piece2.setH(piece2.h + diff)
                            if typecoin2 == 'BLC' or typecoin2 == 'BRC':
                                piece2.setY(piece2.y - diff)
                                piece2.setH(piece2.h + diff)
                    if coin[1] in [self.tlc[1],self.brc[1]] and coin[1] == coin2[1] and (typecoin != 'TRC' and typecoin2 != 'TLC') and coin not in doneList and coin2 not in doneList: #and (coin[0]+coin2[0])>(piece.w+piece2.w) 
                        print('HORIZONTAL')
                        diff = abs(coin2[0]-coin[0])
                        print(piece,piece2,diff)
                        doneList.append(coin)
                        if prio < prio2:
                            if (typecoin == 'BLC' and coin[0] != 650) or (typecoin == 'BRC' and coin[0] == 650):
                                piece.setW(piece.w + diff)
                            if (typecoin == 'BRC' and coin[0] != 150) or (typecoin == 'BLC' and coin[0] == 150):
                                piece.setX(piece.x - diff)
                                piece.setW(piece.w + diff)
                        else:
                            if (typecoin2 == 'BLC' and coin2[0] != 650) or (typecoin2 == 'BRC' and coin2[0] == 650):
                                piece2.setW(piece2.w + diff)  
                            if (typecoin2 == 'BRC' and coin2[0] != 150) or (typecoin2 == 'BLC' and coin2[0] == 150):
                                piece2.setX(piece2.x - diff)  
                                piece2.setW(piece.w + diff)
        print(doneList)
                                
                    
        for piece in self.listePieces:     
            self.drawRoom(piece.getTlc(), piece.getBrc(), piece, 700)
            
        self.can.pack()
    
    def majMatrice(self, piece):
        x = piece.x - self.x
        y = piece.y - self.y
        brc = piece.getBrc()
        print(piece.__str__(), piece.num)
        print([x,y])
        print([piece.w,piece.h])
        print([x+piece.w,y+piece.h])
        
        W = x+piece.w
        H = y+piece.h
        xi = x
        yi = y
        
        self.can.create_rectangle(self.x+self.w+200, self.y, 
                             self.x+self.w*2+200, self.y+self.h)        
        #self.can.create_line((self.x+(self.w)/2)-17, self.y,(self.x+(self.w)/2)+17, self.y, fill = "white")
        self.can.pack()
        
        while xi < W:
            #print(xi)
            yi = y
            while yi < H:
                #print(yi)
                self.mat[xi][yi] = self.mat[xi][yi]*7+piece.num
                self.can.create_text(self.x+xi+self.w+200,self.y+yi,fill=piece.col,font="Arial 1",text=f"{self.mat[xi][yi]}")
                
                yi += 1
            xi += 1
    
    def afficherMatrice(self):
        print(len(self.mat))
        print(self.mat)
        
        return self.mat
    
    def getSurface(self):
        return self.h*self.w
    
    def getCoordoAppart(self):
        return self.x , self.y, self.h, self.w 
    
    def __str__(self):
        return "Appartement"
    
    


    

In [264]:
app = App()

A1 = Appartement(app, 400,500)
#A1.afficherGrille()
A1.afficherAppartement()
A1.createRooms()
A1.placeRooms2()
#wat = A1.afficherMatrice()
#tabModulo(172)

app.mainloop()


Toilettes
TOTALE : [[150, 50, 'TLC'], [650, 50, 'TRC'], [650, 450, 'BRC'], [150, 450, 'BLC']]
COINS : [[150, 50, 'TLC'], [650, 50, 'TRC'], [650, 450, 'BRC'], [150, 450, 'BLC']]
REMOVED : []
Removed corner : [650, 50, 'TRC']
Toilettes placée

Salle de bain
TOTALE : [[150, 50, 'TLC'], [650, 50, 'TRC'], [650, 450, 'BRC'], [150, 450, 'BLC'], [550, 50, 'TRC'], [650, 150, 'TRC']]
COINS : [[150, 50, 'TLC'], [650, 450, 'BRC'], [150, 450, 'BLC'], [550, 50, 'TRC'], [650, 150, 'TRC']]
REMOVED : [[650, 50, 'TRC']]
Removed corner : [150, 450, 'BLC']
Salle de bain placée

Cuisine
TOTALE : [[150, 50, 'TLC'], [650, 50, 'TRC'], [650, 450, 'BRC'], [150, 450, 'BLC'], [550, 50, 'TRC'], [650, 150, 'TRC'], [150, 290, 'BLC'], [275, 450, 'BLC']]
COINS : [[150, 50, 'TLC'], [650, 450, 'BRC'], [550, 50, 'TRC'], [650, 150, 'TRC'], [150, 290, 'BLC'], [275, 450, 'BLC']]
REMOVED : [[650, 50, 'TRC'], [150, 450, 'BLC']]
Removed corner : [650, 450, 'BRC']
Cuisine placée

Chambre
TOTALE : [[150, 50, 'TLC'], [650, 50, '

,0,1,3
0,"[150, 50, TLC]",None,None
1,"[650, 50, TRC]",None,None
2,"[650, 450, BRC]",None,None
3,"[150, 450, BLC]",None,None
4,"[550, 50, TRC]",Toilettes,None
5,"[650, 150, TRC]",Toilettes,None
6,"[150, 290, BLC]",Salle de bain,None
7,"[275, 450, BLC]",Salle de bain,None
8,"[650, 325, BRC]",Cuisine,None
9,"[410, 450, BRC]",Cuisine,None


[150, 50, 'TLC']
[650, 50, 'TRC']
[650, 450, 'BRC']
[150, 450, 'BLC']
[550, 50, 'TRC']
[650, 150, 'TRC']
[150, 290, 'BLC']
[275, 450, 'BLC']
[650, 325, 'BRC']
[410, 450, 'BRC']
[150, 130, 'BLC']
[275, 290, 'BLC']
[275, 165, 'BLC']
[435, 290, 'BLC']
[650, 50, 'TRC']
[150, 450, 'BLC']
[650, 450, 'BRC']
[150, 290, 'BLC']
[275, 290, 'BLC']


C:\Users\idavid\AppData\Local\Temp\ipykernel_15868\4032545478.py:398: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tab['corner'][i] = [elem[0],elem[1]]
C:\Users\idavid\AppData\Local\Temp\ipykernel_15868\4032545478.py:399: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tab['x_corner'][i] = elem[0]
C:\Users\idavid\AppData\Local\Temp\ipykernel_15868\4032545478.py:400: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tab['y_corner'][i] = elem

,1,3,corner,cornertype,x_corner,y_corner
0,None,None,"[150, 50]",TLC,150,50
1,None,None,"[650, 50]",TRC,650,50
2,None,None,"[650, 450]",BRC,650,450
3,None,None,"[150, 450]",BLC,150,450
4,Toilettes,None,"[550, 50]",TRC,550,50
5,Toilettes,None,"[650, 150]",TRC,650,150
6,Salle de bain,None,"[150, 290]",BLC,150,290
7,Salle de bain,None,"[275, 450]",BLC,275,450
8,Cuisine,None,"[650, 325]",BRC,650,325
9,Cuisine,None,"[410, 450]",BRC,410,450


,corner,cornertype,x_corner,y_corner
0,"[650, 50]",TRC,650,50
1,"[150, 450]",BLC,150,450
2,"[650, 450]",BRC,650,450
3,"[150, 290]",BLC,150,290
4,"[275, 290]",BLC,275,290


,1,3,corner_x,cornertype_x,x_corner,y_corner,corner_y,cornertype_y
0,None,None,"[150, 50]",TLC,150,50,NaN,NaN
1,None,None,"[650, 50]",TRC,650,50,"[650, 50]",TRC
2,None,None,"[650, 450]",BRC,650,450,"[650, 450]",BRC
3,None,None,"[150, 450]",BLC,150,450,"[150, 450]",BLC
4,Toilettes,None,"[550, 50]",TRC,550,50,NaN,NaN
5,Toilettes,None,"[650, 150]",TRC,650,150,NaN,NaN
6,Salle de bain,None,"[150, 290]",BLC,150,290,"[150, 290]",BLC
7,Salle de bain,None,"[275, 450]",BLC,275,450,NaN,NaN
8,Cuisine,None,"[650, 325]",BRC,650,325,NaN,NaN
9,Cuisine,None,"[410, 450]",BRC,410,450,NaN,NaN


0 Appartement 1000
1 Toilettes 5
2 Toilettes 5
3 Salle de bain 4
4 Cuisine 3
5 Cuisine 3
6 Chambre 2
7 Salle à manger 1
8 Salle à manger 1


,1,3,corner_x,cornertype_x,prio
7,Salle à manger,None,"[275, 165]",BLC,1
8,Salle à manger,None,"[435, 290]",BLC,1
6,Chambre,None,"[150, 130]",BLC,2
4,Cuisine,None,"[650, 325]",BRC,3
5,Cuisine,None,"[410, 450]",BRC,3
3,Salle de bain,None,"[275, 450]",BLC,4
1,Toilettes,None,"[550, 50]",TRC,5
2,Toilettes,None,"[650, 150]",TRC,5
0,Appartement,None,"[150, 50]",TLC,1000


VERTICAL
Chambre Appartement 80
VERTICAL
Cuisine Toilettes 175
HORIZONTAL
Cuisine Salle de bain 135
HORIZONTAL
Appartement Toilettes 400
[[150, 130], [650, 325], [410, 450], [150, 50]]


In [ ]:
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon

point = Point(0.5, 0.5)
polygon = Polygon([(0, 0), (0, 0.4), (0.2, 0.4), (0.2, 1), (1, 1), (1, 0)])
print(polygon.contains(point))

In [ ]:
plt.plot(*polygon.exterior.xy)

In [ ]:
sizes = [(58, 206), (231, 176), (35, 113), (46, 109)]
positions = rpack.pack(sizes)
positions
poly1 = Polygon(positions)
plt.plot(*poly1.exterior.xy)